### Flash PL bitstream onto FPGA

In [ ]:
from pynq import allocate, Overlay;
import numpy as np;
import matplotlib.pyplot as plot;
from PIL import Image as pil;
import cv2;

# Laad systeemcomponenten in PL

system = Overlay("bd_wrapper.bit");
dma = system.axi_dma_0;
vpu = system.vpu_accel_0;

# Check if the VPU IP core is the right component by chekcing if the registers exist

for reg in ["in_width", "in_height"]:
    assert(hasattr(vpu.register_map, reg));

vpu.register_map

### Load input image

In [ ]:
# Laad input image

image = cv2.imread("0a0a0b1a-7c39d841.png");

plot.imshow(image[:,:,::-1]);

### Allocate SDRAM space & copy image to there

In [ ]:
# Wijs SDRAM toe waar wij de afbeelding willen plaatsen

height, width, channels = image.shape

in_buffer = allocate(shape=(height, width, 3), dtype=np.uint8, cacheable=1)
out_buffer = allocate(shape=(height, width), dtype=np.uint8, cacheable=1)

# Kopieer de afbeelding naar deze ruimte

in_buffer[:] = image

### Activate the HLS Image Processing IP Core

In [ ]:
# Geef een commando aan de digitale logica via de S_AXILITE poort

vpu.register_map.in_width = width
vpu.register_map.in_height = height

dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer)

In [ ]:
vpu.write(0x00, 0x81) # write AP_START and AP_AUTOSTART bits

dma.sendchannel.wait()
dma.recvchannel.wait()

In [ ]:
cv2.imwrite("0a0a0b1a-7c39d841.out.png", out_buffer)
plot.imshow(out_buffer, cmap="gray");
plot.show()
plot.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), cmap="gray")

In [ ]:
del in_buffer
del out_buffer